### Gmarket 베스트 상품 200개 크롤링
- 상품명, 원가, 판매가
- 썸네일 이미지

In [1]:
import requests
from bs4 import BeautifulSoup

In [5]:
response = requests.get("http://corners.gmarket.co.kr/Bestsellers")
dom = BeautifulSoup(response.content, "html.parser") # 파싱

In [20]:
selector = "#gBestWrap > div > div:nth-child(6) > div:nth-child(3) > ul > li"
elements = dom.select(selector)
len(elements)

200

#### 코드, 제목, 판매가

In [20]:
# 코드
elements[0].get("id")

'645747315'

In [19]:
# 제목
elements[0].select_one(".itemname").text

'백설 찹쌀 호떡믹스 6개 골라담기+누르개증정'

In [18]:
# 판매가
elements[0].select_one(".s-price").text.strip().split("\n")[0]

'10,900원'

#### 데이터 프레임으로 만들기

In [21]:
datas = []
for element in elements:
    datas.append({
        "code": element.get("id"),
        "title": element.select_one(".itemname").text,
        "price": element.select_one(".s-price").text.strip().split("\n")[0]
    })
    
items_df = pd.DataFrame(datas)
items_df.tail(2)

,code,price,title
198,1687782101,"81,900원",[지벤]ZB-171S/지벤/6인치안전화/방수내피/다이얼/230~300
199,156577461,"7,900원",BYC/제임스딘外 남녀 내복/내의 기모 발열 남자 여자


## 이미지 다운받기

#### 다운로드 함수 만들기

##### datas 디렉토리 만들기

In [24]:
!mkdir datas

In [25]:
def download(title, link):
    
    response = requests.get(link, stream=True) # stream : 잘게 쪼개서 전송 # 이미지나 동영상에서 많이 사용
    path = "./datas/{}".format(title) # 만든 디렉토리로 경로 설정
    
    with open(path, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

In [28]:
# iterrows() : 인덱스와, 로우를 튜플로 리턴해준다.
for idx, row in items_df.iterrows():
    
    # 이미지 파일을 링크로 만들기
    link = "http://gdimg.gmarket.co.kr/{}/still/300".format(row["code"])
    title = row["title"] + ".png"
    title = title.replace("/", "")
    download(title, link)

In [29]:
imgs = !ls datas
len(imgs)

185